# Project 3 - Machine Learning, Fall 2018

This project is about using a hidden Markov model for gene finding in prokaryotes as explained in class and explored in the exercises in week 46.

We suggest that you do the project in groups of 2-3 students. The project is mandatory and each group must hand in a report describing their work and results as outlined in section Report below. Handin the report in pdf-format via Blackboard no later than **Friday, November 23, 2018, at 16:00**.

### Data set

You are give a data set containing 10
[Staphylococcus](http://en.wikipedia.org/wiki/Staphylococcus) genomes,
each containing several genes (i.e. substring) obeying the "gene syntax" explained in class.
The genomes are between 1.8 million and 2.8 million nucleotides.
For 5 of the genomes, you are also given the location of the genes.
For the remaining 5 genomes, you only know that they contain genes according to the "gene syntax". (You have already explored two of the genomes in the practical exercises in week 45.)

The genomes and their annontations are given in
[FASTA format](http://en.wikipedia.org/wiki/Fasta_format).
There are several Python libraries for reading sequences in this format (use Google to find one),
but it is also easy to read it directly,
and you are welcome to use the simple reader function `read_fasta_file` below.
The data is:

  * The files [genome1.fa](https://users-cs.au.dk/cstorm/courses/ML_e18/projects/handin3/genome1.fa),
    [genome2.fa](https://users-cs.au.dk/cstorm/courses/ML_e18/projects/handin3/genome2.fa),
    [genome3.fa](https://users-cs.au.dk/cstorm/courses/ML_e18/projects/handin3/genome3.fa),
    [genome4.fa](https://users-cs.au.dk/cstorm/courses/ML_e18/projects/handin3/genome4.fa),
    [genome5.fa](https://users-cs.au.dk/cstorm/courses/ML_e18/projects/handin3/genome5.fa),
    [genome6.fa](https://users-cs.au.dk/cstorm/courses/ML_e18/projects/handin3/genome6.fa),
    [genome7.fa](https://users-cs.au.dk/cstorm/courses/ML_e18/projects/handin3/genome7.fa),
    [genome8.fa](https://users-cs.au.dk/cstorm/courses/ML_e18/projects/handin3/genome8.fa),
    [genome9.fa](https://users-cs.au.dk/cstorm/courses/ML_e18/projects/handin3/genome9.fa), and
    [genome10.fa](https://users-cs.au.dk/cstorm/courses/ML_e18/projects/handin3/genome10.fa) contain the 10 genomes.
  * The files [true-ann1.fa](https://users-cs.au.dk/cstorm/courses/ML_e18/projects/handin3/true-ann1.fa),
    [true-ann2.fa](https://users-cs.au.dk/cstorm/courses/ML_e18/projects/handin3/true-ann2.fa),
    [true-ann3.fa](https://users-cs.au.dk/cstorm/courses/ML_e18/projects/handin3/true-ann3.fa),
    [true-ann4.fa](https://users-cs.au.dk/cstorm/courses/ML_e18/projects/handin3/true-ann4.fa), and
    [true-ann5.fa](https://users-cs.au.dk/cstorm/courses/ML_e18/projects/handin3/true-ann5.fa) contain the annotation
    of the first 5 genomes. The annotation is given in FASTA format as a sequence
    over the symbols `N`, `C`, and `R`. The symbol `N`, `C`, or `R` at position _i_ in
    true-ann_k_.fa gives the "state" of the nucleotide at position _i_ in
    genome_k_.fa. `N` means that the nucleotide is non-coding. `C` means that the
    nucleotide is coding and part of a gene in the direction from left to right. `R`
    means that the nucleotide is coding and part of gene in the reverse direction
    from right to left. (This is exactly as in the pratical exercises in week 45.)   
  * All the above files are available in [data-handin3.zip](https://users-cs.au.dk/cstorm/courses/ML_e18/projects/handin3/data-handin3.zip)

By simple reading of the annotations and the corresponding genomes, it is
clear that fx several start-codons are possible (and not only atg as modeled
in class).


In [1]:
def read_fasta_file(filename):
    """
    Reads the given FASTA file f and returns a dictionary of sequences.

    Lines starting with ';' in the FASTA file are ignored.
    """
    sequences_lines = {}
    current_sequence_lines = None
    with open(filename) as fp:
        for line in fp:
            line = line.strip()
            if line.startswith(';') or not line:
                continue
            if line.startswith('>'):
                sequence_name = line.lstrip('>')
                current_sequence_lines = []
                sequences_lines[sequence_name] = current_sequence_lines
            else:
                if current_sequence_lines is not None:
                    current_sequence_lines.append(line)
    sequences = {}
    for name, lines in sequences_lines.items():
        sequences[name] = ''.join(lines)
    return sequences

The function is used as follows. First, we write an example FASTA file:

In [2]:
s = '''
;Example FASTA file
>genomeA
CGATTAAAGA
TAGAAATACA
>annotationA
CCCCCCNNNN
NNNNRRRRRR
'''
with open('test.fa', 'w') as fp:
    fp.write(s)

When we read the file with `read_fasta_file`, we get a dictionary with the keys `genomeA` and `genomeB` corresponding to the contents of the above file.

In [3]:
x = read_fasta_file('test.fa')
x

{'genomeA': 'CGATTAAAGATAGAAATACA', 'annotationA': 'CCCCCCNNNNNNNNRRRRRR'}

In [4]:
print(x['genomeA'])

CGATTAAAGATAGAAATACA


### Problem

Your task is to predict the gene structure of the 5 unannotated genomes, i.e.
[genome6.fa](https://users-cs.au.dk/cstorm/courses/ML_e18/projects/handin3/genome6.fa),
[genome7.fa](https://users-cs.au.dk/cstorm/courses/ML_e18/projects/handin3/genome7.fa),
[genome8.fa](https://users-cs.au.dk/cstorm/courses/ML_e18/projects/handin3/genome8.fa),
[genome9.fa](https://users-cs.au.dk/cstorm/courses/ML_e18/projects/handin3/genome9.fa), and
[genome10.fa](https://users-cs.au.dk/cstorm/courses/ML_e18/projects/handin3/genome10.fa).

As explained in class, predicting gene structure using a HMM involves:

  * Deciding on an initial model structure, i.e. the number of hidden states
    and which transitions and emission should have a fixed probability (e.g. 0
    for "not possible", or 1 for "always the case").
    You might get inspiration from, or even use, one of the model structures discussed in class.
    However, be aware that the models presented in class assume start-codon atg.
    This is not the case in the presented data, where multiple start-codons are possible.

    To predict the full gene structure, you must decide how to handle to fact that
    the genomes have genes in both directions (i.e. a nucleotide can be `C` or `R`).
    You can e.g. choose to make a HMM which only models genes in one direction and
    then use it twice to predict the genes in each direction (beware that start-
    and stop-codons look different in the reversed direction, unless you make the
    reverse-complement of the input string), or you can make a model which models
    genes in both directions.


  * Tune model parameters by training, i.e. set the non-fixed emission and transition probabilities.
    This can be done by several methods as explained in class:
    (1) "By counting" using the 5 genomes with know gene structure because you
    know the underlying hidden state for each observation (i.e. symbol) in these
    sequences, or
    (2) by Viterbi- or EM-training using all the 10 genomes,
    including the 5 genomes with unknown gene structure.
    You can of course also combine the two approaches.
    You can evaluate the performance of your gene finder by computing the
    approximate correlation (AC) between your predictions and
    the true structure using this small python program
    [compare_anns.py](https://users-cs.au.dk/cstorm/courses/ML_e18/projects/handin3/compare_anns.py)
    (see [this paper](http://www.sciencedirect.com/science/article/pii/S0888754396902980)
    for details).

    In this project, training by counting is mandatory, while Viterbi- or
    EM-training is optional. If you implement EM-training you should of course be
    aware of the precision problems of the forward- and backward-algorithms.


  * Use your best model to predict the gene structure for the 5 unannotated
    genomes using the Viterbi algorithm with subsequent backtracking.
    I.e. for each unannotated genome you must find a most likely sequence of
    states in your model generating it, and convert this sequence of states into a
    FASTA file giving the annotation of each nucleotide as `N`, `C`, or `R`.
    You should make files, `pred-ann6.fa`, ..., `pred-ann10.fa`, giving the
    annotation for the 5 unannotated genomes. 
    
    You can use the www-service [GeneFinder Verifier](https://services.birc.au.dk/genefinder-verifier/) to compare your predictions on Genome 6-10 against their true structures.
   

### Report

You (i.e. your group) must hand in a report (in pdf-format) describing your
work and results and a zip-archive of the files `pred-ann6.fa` and
`pred-ann7.fa`,..., `pred-ann10.fa` containing your predictions on the 5
genomes with unknown structure. The files _must_ be in FASTA format similar to
the annotation of the other genomes. Handin the report and zip-archive via
Blackboard no later than **Friday, November 23, 2018, at 16:00**. Your report should
be no more than 3 pages and clearly state who is in the group. It must cover:

  * The status of the work, i.e. does it work, if not, then why.


  * An explanation and illustration of your model structure (i.e. a transition
    diagram). This should include an explanation of how you model that there
    can be several start and stop codons, and an explanation of how you have
    trained your model. For training by counting you should comment on how you have
    translated the given annotations of `C`, `N`, and `R`'s into corresponding sequences
    of hidden states. The transition-diagram should contain all non-0 probabilities
    as computed by your training.


  * An explanation of how you have predicted the gene structure for the unannotated sequences.


  * An evaluation of the performance of your gene predictor.
    You must make a 5-fold cross validation on the 5 genomes with known structure:

    For each genome 1 to 5, you train your model by training-by-counting on the
    remaining 4 genomes, and predict the gene structure on the genome you picked.
    You compute and report the approximate correlation coefficient (AC) between
    your prediction and the true annotation using the small python program
    [compare_anns.py](https://users-cs.au.dk/cstorm/courses/ML_e18/projects/handin3/compare_anns.py).
    Include a table with the computed ACs in your report.
    
    
  * The result of comparing your predictions on the 5 genomes with unknown structure against their true structures via the www-service [GeneFinder Verifer](https://services.birc.au.dk/genefinder-verifier/).

In [4]:
import numpy as np
import math

def read_fasta_file(filename):
    """
    Reads the given FASTA file f and returns a dictionary of sequences.

    Lines starting with ';' in the FASTA file are ignored.
    """
    sequences_lines = {}
    current_sequence_lines = None
    with open(filename) as fp:
        for line in fp:
            line = line.strip()
            if line.startswith(';') or not line:
                continue
            if line.startswith('>'):
                sequence_name = line.lstrip('>')
                current_sequence_lines = []
                sequences_lines[sequence_name] = current_sequence_lines
            else:
                if current_sequence_lines is not None:
                    current_sequence_lines.append(line)
    sequences = {}
    for name, lines in sequences_lines.items():
        sequences[name] = ''.join(lines)
    return sequences




class HMM:
    """
        Make init and transform to np array. 
    """
    def __init__(self, init_probs, trans_probs, emission_probs):
        self.init_probs = np.array(init_probs)
        self.trans_probs = np.array(trans_probs)
        self.emission_probs = np.array(emission_probs)

    def log(self, x):
        if x == 0:
            return float('-inf')
        return math.log(x)

    """
        Make mapping from: 
            obs_example = 'GTTTCCCAGTGTATATCGAGGGATACTACGTGCATAGTAACATCGGCCAA'
        to:
            obs_example_trans= [2,3,3,3,1,1,1,0,2,3,2,3,0,3,0,3,1,2,0,2,2,2,0,3,0,1,3,0,1,2,3,2,1,0,3,0,2,3,0,0,1,0,3,1,2,2,1,1,0,0]
    """
    def translate_observations_to_indices(self, obs):
        mapping = {'a': 0, 'c': 1, 'g': 2, 't': 3}

        return [mapping[symbol.lower()] for symbol in obs]


    def translate_indices_to_observations(self, indices):
        mapping = ['a', 'c', 'g', 't']

        return ''.join(mapping[idx] for idx in indices)


    def translate_path_to_indices(self, path):

        return list(map(lambda x: int(x), path))

    def translate_indices_to_path(self, indices):
        
        return ''.join([str(i) for i in indices])


    """
        validate hmm as follows.
        1. The initial probabilities must sum to 1.
        2. Each row in the matrix of transition probabilities must sum to 1.
        3. Each row in the matrix of emission probabilities must sum to 1.
        4. All numbers should be between 0 and 1, inclusive.
    """
    
    def validate(self):
        # 1. 
        if self.init_probs.sum() != 1:
            return False
        
        # 2.
        row_sums2 = self.trans_probs.sum(axis=1)
        row_ones2 = np.ones(row_sums2.shape)
        if not np.allclose(row_sums2, row_ones2):
            return False
        
        # 3. 
        row_sums3 = self.emission_probs.sum(axis=1)
        row_ones3 = np.ones(row_sums3.shape)
        if not np.allclose(row_sums3, row_ones3):
            return False

        # 4. Are they in range [0:1]
        init_range = np.all((self.init_probs >= 0) & (self.init_probs <= 1))
        trans_range = np.all((self.trans_probs >= 0) & (self.trans_probs <= 1))
        emission_range = np.all((self.emission_probs >= 0) & (self.emission_probs <= 1))
        
        if not (init_range and trans_range and emission_range):
            return False
        
        # We have passed all tests. It is a valid HMM
        return True 


    """
    Returns the log transformed joint probability of x and z
    """
    def joint_prob_log(self, x, z):
        logp = self.log(self.init_probs[z[0]]) + self.log(self.emission_probs[z[0]][x[0]])
        for i in range(1, len(x)):
            logp = logp + self.log(self.trans_probs[z[i-1]][z[i]]) + self.log(self.emission_probs[z[i]][x[i]])
        return logp


    
"""
    ########## HELPER FUNCTIONS ###############3
""" 

def make_table(m, n):
    """
        Make a table with `m` rows and `n` columns filled with zeros.
    """
    return [[0] * n for _ in range(m)]





def main():
    # start-codon + inner + stop-codon = 9+9+3 = 21
    # Coding and reverese coding = 42
    # Init state = 43
    K = 43
    def fromto(source, target):
        return [int(i==target) for i in range(K)]

    
    def from_distribute_to(source, targetlist):
        val = 1.0/len(targetlist)
        result = [0 for i in range(K)]
        for j in range(len(targetlist)):
            result[targetlist[j]] = val
        return result

    init_probs = fromto(0, 0)  #Only start in state 0. Hacky reuse of func :) 

    init_trans_probs = [
        from_distribute_to(0, [1,4,7,34,37,40]),         # 0 -> 1 | 0 -> 4 | 0 -> 7 | 0 -> 34 | 0 -> 37 | 0 -> 40
        #Forward
        #start-codon
        fromto(1,2), fromto(2,3), fromto(3,10),         # 1 -> 2 -> 3 -> 10
        fromto(4,5), fromto(5,6), fromto(6,10),         # 4 -> 5 -> 6 -> 10
        fromto(7,8), fromto(8,9), fromto(9,10),         # 7 -> 8 -> 9 -> 10
        #inner                    
        fromto(10,11),                                  # 10 \> 11 
        fromto(11,12),                                  # 11 \> 12
        from_distribute_to(12, [10, 15, 18, 21]),        # 12 -> 10 | 12 -> 15 | 12 -> 18 | 12 -> 21
        #stop-codon
        fromto(13, 0), fromto(14, 13), fromto(15, 14),  # 0 <- 13 <- 14 <- 15
        fromto(16, 0), fromto(17, 16), fromto(18,17),   # 0 <- 16 <- 17 <- 18
        fromto(19, 0), fromto(20, 19), fromto(21,20),   # 0 <- 19 <- 20 <- 21

        #Reverse
        #stop-codon        
        fromto(22, 0), fromto(23, 22), fromto(24,23),   # 0 <- 22 <- 23 <- 24
        fromto(25, 0), fromto(26, 25), fromto(27,26),   # 0 <- 25 <- 26 <- 27
        fromto(28, 0), fromto(29, 28), fromto(30,29),   # 0 <- 28 <- 29 <- 30
        #inner        
        from_distribute_to(31, [33,30,27,24]),          # 31 -> 33 | 31 -> 30 | 31 -> 27 | 31 -> 24                                  
        fromto(32, 31),                                 # 32 \> 31
        fromto(33, 32),                                 # 33 \> 32
        #start-codon        
        fromto(34, 35), fromto(35, 36), fromto(36, 33), # 34 -> 35 -> 36 -> 33
        fromto(37, 38), fromto(38, 39), fromto(39, 33), # 37 -> 38 -> 39 -> 33
        fromto(40, 41), fromto(41, 42), fromto(42, 33)  # 40 -> 41 -> 42 -> 33
    ]
    

    init_emission_probs = [
        #   A     C     G     T
        [0.25, 0.25, 0.25, 0.25],   #init

        #Forward
        [1, 0, 0, 0],   #start1     1
        [0, 0, 0, 1],   #start1     2
        [0, 0, 1, 0],   #start1     3
        [0, 0, 1, 0],   #start2     4
        [0, 0, 0, 1],   #start2     5
        [0, 0, 1, 0],   #start2     6
        [0, 0, 0, 1],   #start3     7
        [0, 0, 0, 1],   #start3     8
        [0, 0, 1, 0],   #start3     9
        [0.25, 0.25, 0.25, 0.25],   #inner
        [0.25, 0.25, 0.25, 0.25],   #inner
        [0.25, 0.25, 0.25, 0.25],   #inner
        [0, 0, 0, 1],   #stop1      13
        [1, 0, 0, 0],   #stop1      14
        [1, 0, 0, 0],   #stop1      15
        [0, 0, 0, 1],   #stop2      16
        [1, 0, 0, 0],   #stop2      17
        [0, 0, 1, 0],   #stop2      18
        [0, 0, 0, 1],   #stop3      19
        [0, 0, 1, 0],   #stop3      20
        [1, 0, 0, 0],   #stop3      21

        # Reverse
        # A  C  G  T
        [0, 1, 0, 0],   #stop1
        [1, 0, 0, 0],   #stop1
        [0, 0, 0, 1],   #stop1
        [0, 1, 0, 0],   #stop2
        [1, 0, 0, 0],   #stop2
        [0, 1, 0, 0],   #stop2
        [0, 1, 0, 0],   #stop3
        [1, 0, 0, 0],   #stop3
        [1, 0, 0, 0],   #stop3
        [0.25, 0.25, 0.25, 0.25],   #inner
        [0.25, 0.25, 0.25, 0.25],   #inner
        [0.25, 0.25, 0.25, 0.25],   #inner
        [0, 0, 0, 1],   #start1
        [0, 0, 0, 1],   #start1
        [1, 0, 0, 0],   #start1
        [0, 1, 0, 0],   #start2
        [0, 0, 0, 1],   #start2
        [1, 0, 0, 0],   #start2
        [0, 0, 0, 1],   #start3
        [0, 1, 0, 0],   #start3
        [1, 0, 0, 0],   #start3
    ]

    #x = read_fasta_file('genome1.fa')
    
    hmm = HMM(init_probs, init_trans_probs, init_emission_probs)
    isvalid = hmm.validate()
    
    print(isvalid)


if __name__ == '__main__':
    main()

True
